In [6]:
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
#클래스로 다시 구성해서 __init__을 통해서 내부 변수를 다루는 기능 구현 필요,,,

def return_id_list(tag_type_list,year=[-1,-1]):
    '''
    tag_type_list : [a,b] ==> 직무코드별 탐색
    year : [a,b] ==> 경력 a이상b이하, [-1,-1]입력시 모든경력

    1번째문제:
    0번째 게시물부터 100개씩 크롤링 while true
    오류발생! => ex) 총 게시물이 321개인데 300개 크롤링 후 다음100개를 크롤링하려했기때문
    따라서 재귀호출을 통해 크롤링 수를 100개씩 -> 오류발생! -> 10개씩 -> 오류발생! -> 1개씩 크롤링하는 함수구현
    만약 게시물이 321개라면 300개 크롤링 -> 20개 크롤링 - 1개 크롤링 return

    2번째문제:  --중요 ☆☆☆☆☆
    [1024, 1025, 10231, 1634, 655] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어를
    쿼리에 한번에 넣으니 합집합이아닌 교집합만 출력됨
    이를 해결하기 위해 return_id_list에 list형태로 ex)[1024, 1025, 10231, 1634, 655] 값을 넣으면 각 값의 id를 모두 추출하고 중복을 제거하는
    방식으로 변경

    코드 사용 예시:
    tag_type_list = [1024, 1025, 10231, 1634, 655] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어
    id_list = return_id_list(tag_type_list)
    '''

    def crawl_id(tag,year,limit=100,offset=0):
        url = 'http://www.wanted.jobs/api/v4/jobs?'

        params ={
        1656232918453:'',    #사용자번호?
        'country': 'all',
        'tag_type_ids': tag, #직무
        'job_sort': 'job.latest_order',    #최신순 정렬
        'locations': all,
        'years': year[0],    #경력 이상    경력상관없이 검색하려면 -1 , 신입은0
        'years': year[1],    #경력 이하    경력상관없이 검색하려면 -1 , 신입은0
        'limit': limit,    #한 번에 조회 가능한 수 (최대100)
        'offset': offset     #조회할 게시물의 첫 index        ex) limit=100 offset=10  => 10번게시물부터 110번게시물까지 크롤링
        }

        #서버에 url과 쿼리로 요청
        r = requests.get(url,
                        params = params)
        #요청한 데이터 json포멧으로 변환
        r = r.json()
        #json포멧 데이터중 id컬럼만 추출
        id_list = [i['id'] for i in r['data']]
        return id_list

    def crawl_all_id(tag,year,limit=100,offset=0):
        try:
            while True:
                id_list.extend(crawl_id(tag,year,limit,offset))
                offset+=limit
                
        except:
            if limit != 1:
                return crawl_all_id(tag,year,limit/10,offset)
    
    id_list=[]

    for tag in tag_type_list:
        crawl_all_id(tag,year)

    return list(set(id_list))#중복제거

In [8]:
def crawl_job(id_list):
    df_list = []
    
    for id in id_list:
        url = f'https://www.wanted.jobs/api/v4/jobs/{id}?1656259528432'
        r = requests.get(url)
        r = r.json()['job']

        #1개의 게시물 크롤링할때마다 데이터프레임에 append 또는 concat하는것보다
        #list에 append하고 마지막에 한번에 concat하는게 속도가 더 빠르다고 함
        df_list.append(pd.json_normalize(r))
        
    df = pd.concat(df_list, ignore_index=True)
    return df

In [9]:
def engineering(df):
    #사용하지 않을 컬럼 drop
    drop_col = ['is_crossboarder','like_count','is_like','score','company_images','status','is_bookmark','is_company_follow',
            'compare_country','matching_score','short_link','address.geo_location.n_location.address','address.geo_location.location.lat','address.geo_location.location.lng','address.geo_location.viewport.northeast.lat',
            'address.geo_location.viewport.northeast.lng','address.geo_location.viewport.southwest.lat','address.geo_location.viewport.southwest.lng','address.geo_location.bounds','detail.intro','detail.benefits','company.id','company.application_response_stats.level',
            'company.application_response_stats.delayed_count', 'company.application_response_stats.remained_count', 'company.application_response_stats.type', 'logo_img.thumb', 'title_img.origin', 'title_img.thumb', 'reward.formatted_total', 'reward.formatted_recommender', 'reward.formatted_recommendee', 'address.geo_location.bounds.northeast.lat', 'address.geo_location.bounds.northeast.lng', 'address.geo_location.bounds.southwest.lat', 'address.geo_location.bounds.southwest.lng','address.geo_location']
    df = df.drop(drop_col,axis=1)

    #직무별 고유 id dict
    category_tags_dict = {873:'웹 개발자', 872:'서버 개발자', 669:'프론트엔드 개발자',10110:'소프트웨어 엔제니어', 660:'자바 개발자',
                        677:'안드로이드 개발자', 678:'iOS 개발자', 895:'Node.js 개발자', 655:'데이터 엔지니어', 899:'파이썬 개발자',
                        674:'DevOps / 시스템 관리자', 900:'C,C++ 개발자', 665:'시스템,네트워크 관리자', 1634:'머신러닝 엔지니어', 1024:'데이터 사이언티스트',
                        1025:'빅데이터 엔지니어', 676:'QA,테스트 엔지니어', 877:'개발 매니저', 1026:'기술지원', 671:'보안 엔지니어',
                        876:'프로덕트 매니저', 1027:'블록체인 플랫폼 엔지니어', 893:'PHP 개발자', 658:'임베디드 개발자', 939:'웹 퍼블리셔',
                        672:'하드웨어 엔지니어', 10111:'크로스플랫폼 앱 개발자', 661:'.NET 개발자', 896:'영상,음성 엔지니어', 10231:'DBA',
                        898:'그래픽스 엔지니어', 795:'CTO', 10112:'VR 엔지니어', 10230:'ERP전문가', 894:'루비온레일즈 개발자',
                        1022:'BI엔지니어', 793:'CIO'}

    try:
        #skill_tags 비어있는 행 제거
        df = df.loc[df['skill_tags'].apply(lambda x :  x !=[])]
        #skill_tags 문자열에서 키워드만 추출
        df['skill_tags'] = df['skill_tags'].apply(lambda x : [list(i.values())[0] for i  in x])
        #company_tags 문자열에서 키워드만 추출
        df['company_tags'] = df['company_tags'].apply(lambda x : [i['title'] for i  in x])
        #category_tags 직무id만 추출
        df['category_tags'] = df['category_tags'].apply(lambda x : [i['id'] for i  in x])
        

    except:#크롤링시 가끔 dict list가 문자열형태로 존재하는 경우가 발생 ex) '[a,b,c,d]', '{a:1,b:2,c:3}' 이런경우 문자열로접근
        #skill_tags 비어있는 행 제거
        df = df.loc[df['skill_tags'].apply(lambda x :  x !='[]')]
        #skill_tags 문자열에서 키워드만 추출
        df['skill_tags'] = df['skill_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i])
        #company_tags 문자열에서 키워드만 추출
        df['company_tags'] = df['company_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i][:-1])
        #category_tags 직무id만 추출
        df['category_tags'] = df['category_tags'].apply(lambda x : [re.sub(r'[^0-9]', '', v) for i,v in enumerate(x.split(':')) if i % 2 == 0][1:])
        
        #모든 데이터가 문자열로 들어가 있어서 dict의 key를 str로 변경
        category_tags_dict = {str(i):v for i,v in category_tags_dict.items()}
        #0번 컬럼에 index추가되어서 drop
        df = df.iloc[:,1:]

    #고유 id로 게시물 url 컬럼 생성
    df['url'] = df['id'].apply(lambda x : r'https://www.wanted.co.kr/wd/' + str(x))
    #category_tags id 맵핑
    df['category_tags'] = df['category_tags'].apply(lambda x : [category_tags_dict[i] for i in x])


    return df.reset_index(drop=True)


In [10]:
def year_creat(tag_type_list = [1024, 1025, 10231, 1634, 655]):
    '''
    직무list를 넣으면 for i in range(11) 
    경력을 0년 1년 ....10년 각각 크롤링해서 중복 제거 후 dict에 담아서 리턴

    0-5년차를 요구하는 게시물의id는 0-5컬럼에 모두 id 존재하는 문제가 있음

    좀 오래걸림 약 15분
    '''
    year_id_df = {i : set(return_id_list(tag_type_list,year=[i,i])) for i in range(11)}

    year_id_df = pd.DataFrame.from_dict(year_id_df, orient='index').T

    return year_id_df

def year_mapping(year_id_df, df):
    '''
    0-5년차를 요구하는 게시물의id는 0-5컬럼에 모두 id 존재하는 문제가 있음

    df['year'] = [최소경력,최대경력] 으로 구삼
    최소경력은 가장 작은 수가 중요하고 최대경력은 가장 큰 수가 제일 중요하다
    여러 컬럼에 동시에 존재하는 id를 줄여 연산을 최적화하기 위해서
    
    각 컬럼을 set집합 처리하고 최소경력을 위한 increase변수를 보면
    1년차 = 1년차집합-0년차집합 =>0년차에 존재하지않는 1년차원소만 존재하게됨 최소경력은 0년차가 제일 중요하기 떄문
    이렇게 모든 경력을 딕셔너리 컴프리핸션으로 반복하게되면 더이상 모든 컬럼에 중복되는 id가 존재하지 않는다.

    가끔 경력요구사항이 없는 공고가 존재해서 이는 경력무관이라고 판단
    '''
    increase = {i : list(set(year_id_df.iloc[:,i]) - set(year_id_df.iloc[:,i-1])) for i in range(1,11)}
    increase[0] = list(year_id_df.iloc[:,0])
    
    decrease = {i : [set(year_id_df.iloc[:,i]) - set(year_id_df.iloc[:,i+1])] for i in range(9,-1,-1)}
    decrease[10] = list(year_id_df.iloc[:,10])

    df['year'] = [[-1,-1]] * len(df)

    for key in range(0,11):
        #increase에 존재하는 id가 있는 df 검색 후 in 한다면 경력 추가
        df.loc[df['id'].isin(increase[key]),'year'] = df.loc[df['id'].isin(increase[key]),'year'].apply(lambda x : [x[0]+1+key,x[1]] )
        df.loc[df['id'].isin(decrease[key]),'year'] = df.loc[df['id'].isin(decrease[key]),'year'].apply(lambda x : [x[0],x[1]+1+key] )
    

    return df

# year_id_df = year_creat()

# df = year_mapping(year_id_df.iloc[:,1:],df)

데이터셋 설명 노션링크
https://marbled-option-060.notion.site/b136d324d68b4089a9e057d161f96beb?v=11b9b89be2ea47659c102edfba2904a2

In [11]:
#크롤링 실행 구문


               #[데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어, 데이터엔지니어]
tag_type_list = [1024, 1025, 10231, 1634, 655]

#조건직무에 맞는 id 크롤링
id_list = return_id_list(tag_type_list)


#id별 게시물에 접근에서 크롤링
df_job = crawl_job(id_list)

df = engineering(df_job)

#year컬럼 추가 약15분
year_id_dict = year_creat()

df = year_mapping(year_id_dict,df)

df.head()

,id,due_time,skill_tags,company_tags,position,category_tags,address.country,address.full_location,address.geo_location.n_location.lat,address.geo_location.n_location.lng,...,detail.requirements,detail.main_tasks,detail.preferred_points,company.industry_name,company.application_response_stats.avg_rate,company.application_response_stats.avg_day,company.name,logo_img.origin,url,year
0,36864,None,"[Java, Python, SQL]","[Salary Top 1%, Rapid Company Growth, Less tha...",백엔드 개발자 (빅데이터 플랫폼),"[자바 개발자, 서버 개발자, 빅데이터 엔지니어]",Korea,서울 마포구 새창로 7 SNU장학빌딩 16층,37.542485,126.950974,...,• 프레임워크를 사용하지 않고 자바만을 이용한 멀티스레딩 프로그래밍 경험\n• JD...,• 로그프레소 기반의 빅데이터 플랫폼 백엔드 서버 개발\n• 로그프레소 백엔드와 프...,• 1개 이상의 범용 프로그래밍 언어 경험 (스크립트 언어 포함)\n• 컴퓨터공학에...,"IT, Media",83.0,3,로그프레소,https://static.wanted.co.kr/images/wdes/0_4.83...,https://www.wanted.co.kr/wd/36864,"[2, -1]"
1,104449,None,"[C#, C++]","[Salary Top 1%, Rapid Company Growth, 51~300 e...",AI researcher for biology,[머신러닝 엔지니어],Korea,"경기도 성남시 분당구 판교역로 152, 알파돔타워 12층 카카오브레인",None,None,...,"필수조건\n\n- 파이썬 프로그래밍 능숙\n- 딥러닝 프레임워크 (PyTorch, ...",- 본 공고를 통해 영입된 분은 아래 프로젝트들에 참여하여 주도적으로 연구 업무를 ...,우대 조건 (아래 요건 중 최소 3가지 이상의 조건을 보유하신분을 선호합니다)\n\...,"IT, Media",93.0,4,카카오브레인,https://static.wanted.co.kr/images/wdes/0_4.f9...,https://www.wanted.co.kr/wd/104449,"[1, -1]"
2,8195,None,"[Git, Linux, C / C++, C#, Java, Python, C++, U...","[Rapid Company Growth, Less than 50 employees,...",컴퓨터비전/머신러닝 연구원,"[C,C++ 개발자, 데이터 사이언티스트, 머신러닝 엔지니어]",Korea,서울특별시 강남구 강남대로132길 25 4층,37.512706,127.023015,...,- 석사/박사 학위 소유자 선호\n- 컴퓨터비전 프로젝트 유경험자\n- 전문연구요원...,"- 딥러닝 모델 개발 및 경량화\n- 신체 부위 검출기 개발 (얼굴, 눈, 손, 핸...",- 컴퓨터비전 프로젝트 리딩 경험\n- 신체 부위 검출 및 인식기 개발 경험\n- ...,"IT, Media",85.7,8,브이터치(VTouch),https://static.wanted.co.kr/images/wdes/0_4_ab...,https://www.wanted.co.kr/wd/8195,"[1, 10]"
3,96259,None,"[iOS, Swift, MVVM]","[Rapid Company Growth, 51~300 employees, Estab...","DB(DBA, DW)","[데이터 엔지니어, 서버 개발자, 빅데이터 엔지니어]",Korea,압구정로148,37.525203,127.02507,...,- SQL Server DBA 운영 업무 경력 2년/7년 이상 또는 그에 준하는 역...,"- AWS Aurora, MySQL DB 운영 및 기술지원\n- DB 장애 대응 및...",- 이기종 데이터베이스 운영 및 마이그레이션 경험이 있으신 분\n- 대용량 트래픽 ...,"IT, Media",87.6,3,머스트잇,https://static.wanted.co.kr/images/wdes/0_4.0b...,https://www.wanted.co.kr/wd/96259,"[7, 10]"
4,92165,None,"[GraphQL, Hadoop, iOS, MySQL, Pytorch, React, ...","[Salary Top 6~10%, Stable Investment, Rapid Co...","지그재그 데이터 사이언티스트(추천, 개인화)",[데이터 사이언티스트],Korea,서울특별시 강남구 테헤란로 521(파르나스타워) 27층,37.509545,127.060869,...,• 주요 업무의 과제 중 한 가지는 자신 있으신 분\n• 데이터 모델링을 통해 비즈...,• 데이터 모델링을 통한 전사 과제 해결\n• 추천/개인화 모델 고도화\n• 쇼핑몰...,• 프로덕션 레벨의 ML모델 개발 프로세스에 대한 이해가 높고 경험이 있으신 분\n...,"IT, Media",88.4,4,카카오스타일(Kakao Style),https://static.wanted.co.kr/images/wdes/0_4.91...,https://www.wanted.co.kr/wd/92165,"[1, 10]"


## DB 테이블 생성 및 데이터 삽입

In [30]:
import psycopg2

host = 'arjuna.db.elephantsql.com'
user = 'pvltcvea'
password = 'koXbh3YWTCItiJduk1ioNrRBp3eOhlbz'
database = 'pvltcvea'

conn = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS job;")
cur.execute("""
    CREATE TABLE IF NOT EXISTS job(
        id INTEGER NOT NULL PRIMARY KEY,
        skill_tags VARCHAR(1000),
        category_tags VARCHAR(1000),
        requirements VARCHAR(10000),
        main_tasks VARCHAR(10000),
        preferred_points VARCHAR(10000),
        company VARCHAR(1000),
        logo_img VARCHAR(1000),
        url VARCHAR(1000),
        career VARCHAR(100)
    );
""")

In [31]:
for i in range(len(df)):
  id = df.loc[i,'id']
  skill_tags = ' '.join(df.loc[i,'skill_tags'])
  category_tags = ', '.join(df.loc[i,'category_tags'])
  requirements = df.loc[i,'detail.requirements']
  main_tasks = df.loc[i,'detail.main_tasks']
  preferred_points = df.loc[i,'detail.preferred_points']
  name = df.loc[i,'company.name']
  logo_img = df.loc[i,'logo_img.origin']
  url = df.loc[i,'url']
  career = ''
  if df.loc[i,'year'][0] == -1 and df.loc[i,'year'][1] == -1:
    career = '-1'
  elif df.loc[i,'year'][0] == -1:
    for num in range(0, df.loc[i,'year'][1]+1):
      if num > 9:
        break
      career += str(num)
  elif df.loc[i,'year'][1] == -1:
    for num in range(df.loc[i,'year'][0], 10):
      if num > 9:
        break
      career += str(num)
  else:
    for num in range(df.loc[i,'year'][0], df.loc[i,'year'][1]+1):
      if num > 9:
        break
      career += str(num)
  cur.execute("""
    INSERT INTO job (id,skill_tags,category_tags,requirements,main_tasks,preferred_points,company,logo_img,url,career) VALUES (%s,%s,%s
    ,%s,%s,%s,%s,%s,%s,%s);
  """, (int(id),skill_tags,category_tags,requirements,main_tasks,preferred_points,name,logo_img,url,career))

conn.commit()

cur.close()
conn.close()